In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from ngboost import NGBClassifier

In [3]:
# 全量数据

train_data = pd.read_csv('./data_format1/train_format1.csv')
test_data = pd.read_csv('./data_format1/test_format1.csv')
user_info = pd.read_csv('./data_format1/user_info_format1.csv')
user_log = pd.read_csv('./data_format1/user_log_format1.csv')


In [4]:
# 加载样本
'''
train_data = pd.read_csv('./sample/train_data_sample.csv')
test_data = pd.read_csv('./sample/test_data_sample.csv')
user_info = pd.read_csv('./sample/user_info_sample.csv')
user_log = pd.read_csv('./sample/user_log_sample.csv')
'''

"\ntrain_data = pd.read_csv('./sample/train_data_sample.csv')\ntest_data = pd.read_csv('./sample/test_data_sample.csv')\nuser_info = pd.read_csv('./sample/user_info_sample.csv')\nuser_log = pd.read_csv('./sample/user_log_sample.csv')\n"

In [5]:
user_log.rename(columns={'seller_id': 'merchant_id'}, inplace=True)

In [6]:
# 数据清洗
user_log['brand_id'].fillna(0, inplace=True)
user_log['brand_id'] = user_log['brand_id'].astype('int64')
user_log['time_stamp'] = pd.to_datetime(user_log['time_stamp'], format='%H%M')
user_info['age_range'].fillna(0, inplace=True)
user_info = user_info.replace([8], [7])
user_info['age_range'] = user_info['age_range'].astype('int64')
user_info['gender'].fillna(2, inplace=True)
user_info['gender'] = user_info['gender'].astype('int64')

In [7]:
print(train_data.info())
print(user_info.info())
print(user_log.info(null_counts=True))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260864 entries, 0 to 260863
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype
---  ------       --------------   -----
 0   user_id      260864 non-null  int64
 1   merchant_id  260864 non-null  int64
 2   label        260864 non-null  int64
dtypes: int64(3)
memory usage: 6.0 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424170 entries, 0 to 424169
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   user_id    424170 non-null  int64
 1   age_range  424170 non-null  int64
 2   gender     424170 non-null  int64
dtypes: int64(3)
memory usage: 9.7 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54925330 entries, 0 to 54925329
Data columns (total 7 columns):
 #   Column       Non-Null Count     Dtype         
---  ------       --------------     -----         
 0   user_id      54925330 non-null  int64         
 1   item_id      549253

In [8]:
train_data['origin'] = 'train'
test_data['origin'] = 'test'
matrix = pd.concat([train_data, test_data], ignore_index=True, sort=False)
matrix.drop('prob', axis=1, inplace=True)
matrix = matrix.merge(user_info, on='user_id', how='left')

In [9]:
# 用户特征处理
groups = user_log.groupby(['user_id'])
temp = groups.size().reset_index().rename(columns={0: 'u1'})
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['item_id'].agg([('u2', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['cat_id'].agg([('u3', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['merchant_id'].agg([('u4', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['brand_id'].agg([('u5', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['time_stamp'].agg([('F_time', 'min'), ('L_time', 'max')]).reset_index()
temp['u6'] = (temp['L_time'] - temp['F_time']).dt.seconds/3600
matrix = matrix.merge(temp[['user_id', 'u6']], on='user_id', how='left')
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0: 'u7', 1: 'u8', 2: 'u9', 3: 'u10'})
matrix = matrix.merge(temp, on='user_id', how='left')

# 商户特征处理
groups = user_log.groupby(['merchant_id'])
temp = groups.size().reset_index().rename(columns={0: 'm1'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
temp = groups['user_id', 'item_id', 'cat_id', 'brand_id'].nunique().reset_index().rename(columns={'user_id': 'm2', 'item_id': 'm3', 'cat_id': 'm4', 'brand_id': 'm5'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0: 'm6', 1: 'm7', 2: 'm8', 3: 'm9'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
temp = train_data[train_data['label'] == -1].groupby('merchant_id').size().reset_index().rename(columns={0: 'm10'})
matrix = matrix.merge(temp, on='merchant_id', how='left')

/Users/paulhwang/anaconda3/envs/python3/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


In [10]:
groups = user_log.groupby(['user_id', 'merchant_id'])
temp = groups.size().reset_index().rename(columns={0: 'um1'})
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['item_id', 'cat_id', 'brand_id'].nunique().reset_index().rename(columns={'item_id': 'um2', 'cat_id': 'um3', 'brand_id': 'um4'})
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0: 'um5', 1: 'um6', 2: 'um7', 3: 'um8'})
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['time_stamp'].agg([('first', 'min'), ('last', 'max')]).reset_index()
temp['um9'] = (temp['last'] - temp['first']).dt.seconds/3600
temp.drop(['last', 'first'], axis=1, inplace=True)
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')

/Users/paulhwang/anaconda3/envs/python3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  after removing the cwd from sys.path.


In [11]:
matrix['r1'] = matrix['u9'] / matrix['u7']
matrix['r2'] = matrix['m8'] / matrix['m6']
matrix['r3'] = matrix['um7'] / matrix['um5']
matrix.fillna(0, inplace=True)
temp = pd.get_dummies(matrix['age_range'], prefix='age')
matrix = pd.concat([matrix, temp], axis=1)
temp = pd.get_dummies(matrix['gender'], prefix='gen')
matrix = pd.concat([matrix, temp], axis=1)
matrix.drop(['age_range', 'gender'], axis=1, inplace=True)

In [12]:
train_data = matrix[matrix['origin'] == 'train'].drop('origin', axis=1)
test = matrix[matrix['origin'] == 'test'].drop(['origin', 'label'], axis=1)
x_train, y_train = train_data.drop('label', axis=1), train_data['label']

In [15]:
train_x, vali_x, train_y, vali_y = train_test_split(x_train, y_train, test_size=0.2)
model = NGBClassifier(n_estimators=1000, learning_rate=0.01, verbose=True, verbose_eval=50)
model.fit(train_x, train_y.apply(int))

[iter 0] loss=0.2289 val_loss=0.0000 scale=1.0000 norm=2.0000
[iter 50] loss=0.2246 val_loss=0.0000 scale=1.0000 norm=1.9553
[iter 100] loss=0.2227 val_loss=0.0000 scale=1.0000 norm=1.9524
[iter 150] loss=0.2214 val_loss=0.0000 scale=1.0000 norm=1.9593
[iter 200] loss=0.2206 val_loss=0.0000 scale=1.0000 norm=1.9653
[iter 250] loss=0.2200 val_loss=0.0000 scale=1.0000 norm=1.9705
[iter 300] loss=0.2195 val_loss=0.0000 scale=1.0000 norm=1.9745
[iter 350] loss=0.2191 val_loss=0.0000 scale=2.0000 norm=3.9550
[iter 400] loss=0.2187 val_loss=0.0000 scale=1.0000 norm=1.9800
[iter 450] loss=0.2185 val_loss=0.0000 scale=1.0000 norm=1.9824
[iter 500] loss=0.2182 val_loss=0.0000 scale=1.0000 norm=1.9838
[iter 550] loss=0.2180 val_loss=0.0000 scale=1.0000 norm=1.9843
[iter 600] loss=0.2178 val_loss=0.0000 scale=1.0000 norm=1.9847
[iter 650] loss=0.2176 val_loss=0.0000 scale=1.0000 norm=1.9848
[iter 700] loss=0.2174 val_loss=0.0000 scale=1.0000 norm=1.9849
[iter 750] loss=0.2172 val_loss=0.0000 scal

NGBClassifier(Base=DecisionTreeRegressor(ccp_alpha=0.0,
                                         criterion='friedman_mse', max_depth=3,
                                         max_features=None, max_leaf_nodes=None,
                                         min_impurity_decrease=0.0,
                                         min_impurity_split=None,
                                         min_samples_leaf=1,
                                         min_samples_split=2,
                                         min_weight_fraction_leaf=0.0,
                                         presort='deprecated',
                                         random_state=None, splitter='best'),
              Dist=<class 'ngboost.distns.categorical.k_categorical.<locals>.Categorical'>,
              Score=<class 'ngboost.scores.LogScore'>, learning_rate=0.01,
              minibatch_frac=1.0, n_estimators=1000, natural_gradient=True,
              random_state=RandomState(MT19937) at 0x10ACC29E0, tol=0.0

In [16]:
from sklearn.metrics import mean_squared_error
vali = model.predict(vali_x)
mse = mean_squared_error(vali, vali_y)
print('MSE: ', mse)
prob = model.predict_proba(test)
test_data['prob'] = pd.Series(prob[:, 1])
test_data.drop(['origin'], axis=1, inplace=True)

MSE:  0.0629636018630326


In [17]:
test_data.to_csv('prediction_ngboost.csv', index=False)

,user_id,merchant_id,prob
0,163968,4605,0.048878
1,360576,1581,0.080782
2,98688,1964,0.050594
3,98688,3645,0.034064
4,295296,3361,0.065510
...,...,...,...
261472,228479,3111,0.075131
261473,97919,2341,0.024572
261474,97919,3971,0.110609
261475,32639,3536,0.039919
